In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords  
from bs4 import BeautifulSoup 
import tensorflow as tf
import keras
from keras import callbacks


In [2]:
pd.set_option("display.max_colwidth", 200)

In [3]:
data=pd.read_csv("./Reviews.csv",nrows=100000)

In [4]:
len(data)

100000

In [5]:
data.drop_duplicates(subset=['Text'],inplace=True)

In [6]:
data.dropna(axis=0,inplace=True)

In [7]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [8]:
data['Text'][:10]


0    I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labr...
1             Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
2    This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with ...
3    If you are looking for the secret ingredient in Robitussin I believe I have found it.  I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda.  The fl...
4                                                               Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taff

In [9]:
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t))

In [10]:
data['Summary'][:10]

0                            Good Quality Dog Food
1                                Not as Advertised
2                            "Delight" says it all
3                                   Cough Medicine
4                                      Great taffy
5                                       Nice Taffy
6    Great!  Just as good as the expensive brands!
7                           Wonderful, tasty taffy
8                                       Yay Barley
9                                 Healthy Dog Food
Name: Summary, dtype: object

In [11]:
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString

#Call the above function
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(summary_cleaner(t))

data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary
data['cleaned_summary'].replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [12]:
data['cleaned_summary'] = data['cleaned_summary'].apply(lambda x : '_START_ '+ x + ' _END_')

In [13]:
for i in range(5):
    print("Review:",data['cleaned_text'][i])
    print("Summary:",data['cleaned_summary'][i])
    print("\n")

Review: bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better
Summary: _START_ good quality dog food  _END_


Review: product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo
Summary: _START_ not as advertised  _END_


Review: confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story lewis lion witch wardrobe treat seduces edmund selling brother sisters witch
Summary: _START_ delight says it all  _END_


Review: looking secret ingredient robitussin believe found got addition root beer extract ordered made cherry soda flavor medicinal
Summary: _START_ cough medicine  _END_


Review: great taffy great price wide assortment yummy taffy delivery quick taffy lo

In [14]:
max_len_text=80 
max_len_summary=10

In [15]:
# from sklearn.model_selection import train_test_split
# x_tr,x_val,y_tr,y_val=train_test_split(data['cleaned_text'],data['cleaned_summary'],test_size=0.1,random_state=0,shuffle=True)
x_tr=data['cleaned_text']
y_tr=data['cleaned_summary']

In [16]:
from transformers import BartTokenizer, TFBartForConditionalGeneration, BartConfig

In [17]:
model = TFBartForConditionalGeneration.from_pretrained('facebook/bart-large')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [18]:
train_x = tokenizer(list(x_tr),truncation=True,padding="max_length",return_tensors='tf')
train_y = tokenizer(list(y_tr),truncation=True,padding="max_length",return_tensors='tf')


In [34]:

es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [20]:
train_y['input_ids']

<tf.Tensor: shape=(88352, 1024), dtype=int32, numpy=
array([[   0, 1215, 4014, ...,    1,    1,    1],
       [   0, 1215, 4014, ...,    1,    1,    1],
       [   0, 1215, 4014, ...,    1,    1,    1],
       ...,
       [   0, 1215, 4014, ...,    1,    1,    1],
       [   0, 1215, 4014, ...,    1,    1,    1],
       [   0, 1215, 4014, ...,    1,    1,    1]])>

In [44]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [52]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(data['cleaned_text'],data['cleaned_summary'],test_size=0.1,random_state=0,shuffle=True) 

In [53]:
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=1024, padding='post') 
x_val   =   pad_sequences(x_val, maxlen=1024, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [54]:
#preparing a tokenizer for summary on training data 
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

#convert summary sequences into integer sequences
y_tr    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr, maxlen=1024, padding='post')
y_val   =   pad_sequences(y_val, maxlen=1024, padding='post')


In [55]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [57]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history=model.fit([x_tr,y_tr], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1) ,epochs=50,callbacks=[es],batch_size=512, validation_data=([x_val,y_val], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)))

Epoch 1/50


ResourceExhaustedError:  OOM when allocating tensor with shape[512,1,1024,1024] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node tf_bart_for_conditional_generation/model/decoder/Tile_1
 (defined at C:\Users\Finally\anaconda3\lib\site-packages\transformers\models\bart\modeling_tf_bart.py:106)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_368544]

Errors may have originated from an input operation.
Input Source operations connected to node tf_bart_for_conditional_generation/model/decoder/Tile_1:
In[0] tf_bart_for_conditional_generation/model/decoder/strided_slice_3:	
In[1] tf_bart_for_conditional_generation/model/decoder/Tile_1/multiples:

Operation defined at: (most recent call last)
>>>   File "C:\Users\Finally\anaconda3\lib\runpy.py", line 194, in _run_modul

In [43]:
with tf.device("/gpu:0"):
    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
    preprocessed_x = tf.convert_to_tensor(train_x['input_ids'])[:10]
    preprocessed_y = tf.convert_to_tensor(train_y['input_ids'])[:10]
    preprocessed_y = preprocessed_y.reshape(preprocessed_y.shape[0],preprocessed_y.shape[1],1)
    print(preprocessed_y.shape[0])
    print(preprocessed_y.shape[1])
    model.fit(preprocessed_x,preprocessed_y,epochs=5)


10
1024
Epoch 1/5


InvalidArgumentError:  assertion failed: [Condition x == y did not hold element-wise:] [x (sparse_categorical_crossentropy_3/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [10 1024 1] [y (sparse_categorical_crossentropy_3/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [10 16 1024]
	 [[node sparse_categorical_crossentropy_3/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
 (defined at C:\Users\Finally\anaconda3\lib\site-packages\keras\backend.py:5113)
]] [Op:__inference_train_function_292927]

Errors may have originated from an input operation.
Input Source operations connected to node sparse_categorical_crossentropy_3/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert:
In[0] sparse_categorical_crossentropy_3/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/All:	
In[1] sparse_categorical_crossentropy_3/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert/data_0:	
In[2] sparse_categorical_crossentropy_3/SparseSoftmaxCrossEntropyWi